In [33]:
using Pkg
Pkg.activate("/home/fedflorio/phd")

  Activating project at `~/phd`


In [34]:
using LegendrePolynomials, TensorTrains, TensorCast, Tullio, Random, IterTools
using Revise
include("BAMPS.jl")

In [ ]:
d = 4       # bond dimension
T = 4       # length of tensor train
q = (5, )   # number of values for the variables
n = 3       # number of "neighbours"
K = 7       # number of basis elements

AA = [rand_tt(d, T, q...) for i=1:n]

3-element Vector{TensorTrain{Float64, 3}}:
 TensorTrain{Float64, 3}([[0.5137634957257836 0.10376658350948909 0.18324950856689604 0.014606313589829112;;; 0.542127550957072 0.64222863659681 0.6062710539593097 0.4804379948015779;;; 0.1825552787491087 0.17470307219954684 0.9464516723184585 0.532458476992958;;; 0.15098102574334504 0.7549542781818246 0.26100829782719215 0.10883264887002175;;; 0.09204532212403427 0.7067116591072531 0.18362757583182 0.06442704704105118], [0.9306359083018657 0.40655635005777735 0.5460697232169185 0.8476386252128834; 0.13351619434139528 0.001400671578837609 0.9373303973451756 0.9980455935350679; 0.6094818216518173 0.12571837748231718 0.654481909814408 0.026773331159753888; 0.9628281517731789 0.5493849333572556 0.3098134536806214 0.5511803105821779;;; 0.5952416349027304 0.5800689381111108 0.6639864056928354 0.05730890050458037; 0.8533403533904833 0.22944606266551504 0.15807533948417163 0.7961524512480451; 0.16127717928918595 0.2511353596865171 0.41185935909002824

In [36]:
# traj_xᵢᵗ = collect(1:q[1])
# traj_xᵢ = collect(IterTools.product(traj_xᵢᵗ, traj_xᵢᵗ, traj_xᵢᵗ, traj_xᵢᵗ))

# d_y = [1; fill(d, T-1); 1]
# q_y = n.*q
# Py = zeros(q_y[1], q_y[1], q_y[1], q_y[1]) 

# for x1 in traj_xᵢ, x2 in traj_xᵢ, x3 in traj_xᵢ
#     y = x1.+x2.+x3
#     Py[y...] += evaluate(AA[1],x1) * evaluate(AA[2],x2) * evaluate(AA[3],x3)
# end

In [ ]:
function aggregate(AA)
    function tensorprod(A₁, A₂)
        B = map(zip(A₁, A₂)) do (A₁ᵗ, A₂ᵗ)
            b = zeros(size(A₁ᵗ,1), size(A₂ᵗ,1), size(A₁ᵗ,2), size(A₂ᵗ,2), size(A₁ᵗ,3)+size(A₂ᵗ,3))
            @tullio b[m1,m2,n1,n2,y] = A₁ᵗ[m1,n1,x₁] * A₂ᵗ[m2,n2,x₂] * (x₁+x₂==y)
            @cast _[(m1,m2),(n1,n2),y] := b[m1,m2,n1,n2,y]
        end
    end

    B = [copy(Aᵗ) for Aᵗ in AA[1]]
    for i in 2:length(AA)
        B = tensorprod(B,[A for A in AA[i]])
    end

    return TensorTrain(B)
end
;

aggregate (generic function with 1 method)

In [ ]:
Py = aggregate(AA)

TensorTrain{Float64, 3}([[0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0;;; 0.003850938214861605 0.0007777872604549576 … 0.07361435869861967 0.00586759777024823;;; … ;;; 0.15043510820543515 0.8055263987994317 … 0.31678593157971124 0.1394744781810121;;; 0.051360757414926655 0.30549199207572636 … 0.1001020480619327 0.0369780997818634;;; 0.020230003770180003 0.15532326030544483 … 0.019756549353320732 0.00693172650561494], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0011174324968025097 0.00048816005624040303 … 0.054745412844257615 0.0849787573034603; 0.00016031547147016026 1.6818133983211195e-6 … 0.09397067332691085 0.10005758556543354; … ; 0.10321721777863978 0.021290710709305834 … 0.4254767631837125 0.017405263782963833; 0.16305727175855156 0.09303966467277863 … 0.20140881434015143 0.3583206975783332;;; … ;;; 1.2479751647082966 0.4010896009649753 … 0.049288098069421804 0.192

In [ ]:
Px_basis = [BAMPS(AA[i], K) for i in 1:n]

BasisTensorTrain{Float64, 3}([[1.0475593734325286 1.6845859020028355 1.5419227806333204 0.8490672931183891;;; -0.7560245206504468 0.807483964771962 -0.21096635898391133 -0.16654318302233373;;; 0.6765576150655407 0.8672211948340347 -0.3395632733945173 -0.41244750834222943], [2.5806207650553223 1.0986838141147406 0.9903059967501491 1.4795676488815523; 1.852293969009086 0.6592794960110062 1.6088758892611161 2.5279944712250275; 1.2938888449002346 1.423164028744457 1.964871001065684 1.4752708808911075; 2.0475365912111387 1.6695732455839627 1.6861109504467462 1.1848671319769197;;; -0.45542851193378475 -0.4699121225288179 -0.9168306518905177 -0.6938441148513234; -0.1319565165593726 0.45098067631430744 -0.34965831314065265 -0.4559286960533445; -0.21945783443756012 0.7404792784035462 0.18213307298330028 -0.0856431938563833; -0.8546151121759998 -0.11185858938225013 0.4214151677091874 -1.0214542391085142;;; 1.2221967636777564 0.6827256566041745 0.8686785988134976 1.1199590835479205; -0.5771808649